In [ ]:
import backtrader as bt
import setup_path

class TurtleLong(bt.Strategy):
    """
    海龜交易法 - 只做多版本 (Backtrader)
    
    規則摘要：
    - 入場：
        系統1：收盤價突破過去 20 日最高價 → 建多單
        系統2：收盤價突破過去 55 日最高價 → 建多單
        (任一系統給出多頭訊號即可，只要目前沒有持倉)
    - 出場：
        系統1：跌破過去 10 日最低價 → 全部出場
        系統2：跌破過去 20 日最低價 → 全部出場
        以及：跌破「入場價 - 2N」的停損價 → 全部出場
    - N 值：ATR(20)
    - 部位大小：單筆風險 = 帳戶淨值 * risk_per_trade
                  停損距離 = 2N
    - 加碼：每漲 0.5N 加碼 1 單位，最多 max_units 單位
    - 僅做多，不做空
    """

    params = dict(
        # 趨勢參數
        sys1_entry=20,   # 系統1突破 (高點)
        sys1_exit=10,    # 系統1出場 (低點)
        sys2_entry=55,   # 系統2突破 (高點)
        sys2_exit=20,    # 系統2出場 (低點)

        # 波動與風險控管
        atr_period=20,
        risk_per_trade=0.01,   # 單筆風險佔帳戶淨值比例 (1%)
        atr_mult_stop=2.0,     # 停損距離 = 2N
        atr_step_pyramid=0.5,  # 每 0.5N 加碼一次
        max_units=4,           # 單一商品最多單位

        point_value=1.0,       # 每一價格單位的價值 (股票通常 = 1；期貨要自己設)
    )

    def __init__(self):
        self.atr = bt.ind.ATR(self.data, period=self.p.atr_period)

        self.s1_high = bt.ind.Highest(self.data.high(-1), period=self.p.sys1_entry)
        self.s1_low_exit = bt.ind.Lowest(self.data.low(-1), period=self.p.sys1_exit)

        self.s2_high = bt.ind.Highest(self.data.high(-1), period=self.p.sys2_entry)
        self.s2_low_exit = bt.ind.Lowest(self.data.low(-1), period=self.p.sys2_exit)

        self.order = None
        self.units = 0
        self.entry_price = None
        self.stop_price = None
        self.next_add_price = None

    # 計算每個 unit 的張數/股數
    def _calc_unit_size(self):
        # 使用目前帳戶淨值計算 1% 風險
        value = self.broker.getvalue()
        risk_amount = value * self.p.risk_per_trade

        N = self.atr[0]
        if N is None or N <= 0:
            return 0

        # 單位風險（每股/每口損失）= 停損距離 * 每點價值
        unit_risk = self.p.atr_mult_stop * N * self.p.point_value

        if unit_risk <= 0:
            return 0

        size = int(risk_amount / unit_risk)
        return max(size, 0)

    def log(self, txt):
        dt = self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} - {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 仍在撮合中
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Size: {order.executed.size}, Cost: {order.executed.value:.2f}'
                )
            elif order.issell():
                self.log(
                    f'SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Size: {order.executed.size}, Cost: {order.executed.value:.2f}'
                )

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # 無論如何，清掉掛單狀態
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(
            f'TRADE PROFIT, Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}'
        )

    def next(self):
        if self.order:
            # 有掛單在路上就先不動
            return

        close = self.data.close[0]
        N = self.atr[0]

        # 1) 沒有持倉 → 找入場機會
        if not self.position:
            self.units = 0
            self.entry_price = None
            self.stop_price = None
            self.next_add_price = None

            # 只要任一系統出現「向上突破」訊號就建立多單
            long_signal_s1 = close > self.s1_high[0]
            long_signal_s2 = close > self.s2_high[0]

            if (long_signal_s1 or long_signal_s2) and N is not None and N > 0:
                size = self._calc_unit_size()
                if size <= 0:
                    return  # 資金或 N 不適合開倉

                # 建立第一個 unit
                self.order = self.buy(size=size)
                self.units = 1
                self.entry_price = close

                # 初始停損價：入場價 - 2N
                self.stop_price = self.entry_price - self.p.atr_mult_stop * N

                # 下一次加碼價：入場價 + 0.5N
                self.next_add_price = self.entry_price + self.p.atr_step_pyramid * N

                self.log(
                    f'ENTRY LONG, Price: {close:.2f}, Size: {size}, '
                    f'N: {N:.2f}, Stop: {self.stop_price:.2f}, '
                    f'Next Add: {self.next_add_price:.2f}'
                )

        # 2) 已經有多頭部位 → 管理出場 & 加碼
        else:
            # 出場條件：
            # a) 跌破停損價
            # b) 系統1：跌破 10 日低點
            # c) 系統2：跌破 20 日低點
            exit_s1 = close < self.s1_low_exit[0]
            exit_s2 = close < self.s2_low_exit[0]
            exit_stop = self.stop_price is not None and close < self.stop_price

            if exit_stop or exit_s1 or exit_s2:
                self.log(
                    f'EXIT LONG, Price: {close:.2f}, '
                    f'Stop: {self.stop_price if self.stop_price is not None else "nan"}, '
                    f'S1Low: {self.s1_low_exit[0]:.2f}, '
                    f'S2Low: {self.s2_low_exit[0]:.2f}'
                )
                self.order = self.close()
                self.units = 0
                self.entry_price = None
                self.stop_price = None
                self.next_add_price = None
                return  # 出場後本 bar 不再加碼

            # 加碼邏輯：只在順勢 & 未達 max_units 時加碼
            if (
                self.units > 0
                and self.units < self.p.max_units
                and self.next_add_price is not None
                and N is not None
                and N > 0
            ):
                # 價格已經走到加碼價以上 → 再買一個 unit
                if close >= self.next_add_price:
                    size = self._calc_unit_size()
                    if size > 0:
                        self.order = self.buy(size=size)
                        self.units += 1

                        # 設定下一階加碼價：在上一階基礎上再 +0.5N
                        self.next_add_price = self.next_add_price + self.p.atr_step_pyramid * N

                        self.log(
                            f'PYRAMID ADD, Price: {close:.2f}, Size: {size}, '
                            f'Units: {self.units}, Next Add: {self.next_add_price:.2f}'
                        )

            # （進階：也可以選擇將停損價隨趨勢上移，如：不低於最近的 X 日低點，這裡先用固定 2N 停損即可）

/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_85483/3574365615.py:208: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', group_by='ticker', start='2020-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Starting Portfolio Value: 100000.00
2020-04-14 - ENTRY LONG, Price: 69.42, Size: 153, N: 3.26, Stop: 62.90, Next Add: 71.06
2020-04-15 - BUY EXECUTED, Price: 68.30, Size: 153, Cost: 10449.75
2020-04-30 - PYRAMID ADD, Price: 71.06, Size: 189, Units: 2, Next Add: 72.38
2020-05-01 - BUY EXECUTED, Price: 69.23, Size: 189, Cost: 13084.49
2020-05-06 - PYRAMID ADD, Price: 72.71, Size: 200, Units: 3, Next Add: 73.64
2020-05-07 - BUY EXECUTED, Price: 73.33, Size: 200, Cost: 14666.87
2020-05-08 - PYRAMID ADD, Price: 75.21, Size: 211, Units: 4, Next Add: 74.85
2020-05-11 - BUY EXECUTED, Price: 74.72, Size: 211, Cost: 15765.14
2020-07-23 - EXIT LONG, Price: 90.06, Stop: 62.89543219393385, S1Low: 91.06, S2Low: 85.19
2020-07-24 - SELL EXECUTED, Price: 88.26, Size: -753, Cost: 53966.26
2020-07-24 - TRADE PROFIT, Gross: 12493.77, Net: 12373.34
2020-07-31 - ENTRY LONG, Price: 103.08, Size: 208, N: 2.70, Stop: 97.68, Next Add: 104.42
2020-08-03 - BUY EXECUTED, Price: 104.96, Size: 208, Cost: 21831.03
20

In [14]:
# 簡單示範如何在程式裡使用這個策略
if __name__ == '__main__':
    import datetime
    import backtrader as bt
    import yfinance as yf

    cerebro = bt.Cerebro()

    df = yf.download('AAPL', group_by='ticker', start='2020-01-01', end='2025-01-01')
    df = df['AAPL']    # 解 MultiIndex
    datafeed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(datafeed)

    cerebro.addstrategy(TurtleLong)

    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_85483/2165233629.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', group_by='ticker', start='2020-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 100000.00
2020-04-14 - ENTRY LONG, Price: 69.42, Size: 153, N: 3.26, Stop: 62.90, Next Add: 71.06
2020-04-15 - BUY EXECUTED, Price: 68.30, Size: 153, Cost: 10449.75
2020-04-30 - PYRAMID ADD, Price: 71.06, Size: 189, Units: 2, Next Add: 72.38
2020-05-01 - BUY EXECUTED, Price: 69.23, Size: 189, Cost: 13084.49
2020-05-06 - PYRAMID ADD, Price: 72.71, Size: 200, Units: 3, Next Add: 73.64
2020-05-07 - BUY EXECUTED, Price: 73.33, Size: 200, Cost: 14666.87
2020-05-08 - PYRAMID ADD, Price: 75.21, Size: 211, Units: 4, Next Add: 74.85
2020-05-11 - BUY EXECUTED, Price: 74.72, Size: 211, Cost: 15765.14
2020-07-23 - EXIT LONG, Price: 90.06, Stop: 62.89543219393385, S1Low: 91.06, S2Low: 85.19
2020-07-24 - SELL EXECUTED, Price: 88.26, Size: -753, Cost: 53966.26
2020-07-24 - TRADE PROFIT, Gross: 12493.77, Net: 12373.34
2020-07-31 - ENTRY LONG, Price: 103.08, Size: 208, N: 2.70, Stop: 97.68, Next Add: 104.42
2020-08-03 - BUY EXECUTED, Price: 104.96, Size: 208, Cost: 21831.03
20